In [2]:
!pip install fuzzysearch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fuzzysearch: filename=fuzzysearch-0.7.3-cp310-cp310-linux_x86_64.whl size=293923 sha256=40c9c4e19c558897039fcedf23089577d2834f87f6cac84562e424d9d0bcc086
  Stored in directory: /root/.cache/pip/wheels/80/fb/bf/1c4f359d4b13bbc0e2cef8703d8a7c10dcd1e377496c19e6dc
Successfully built fuzzysearch


In [3]:
!pip install fuzzywuzzy

In [40]:
import requests
import json
from datetime import datetime
import pandas as pd
from fuzzywuzzy import fuzz
import plotly.express as px

## Coletando dados usando a url da pesquisa

In [38]:
def coletar_da_url(url:str, palavras_chave:list) -> tuple:
  # Requisita a url
  req = requests.get(url)
  texto_html = req.text
  # Quebra o texto no ponto da página que possui as informações em formato json
  resultado = texto_html.split('window.__PRELOADED_STATE__ =\n')[-1]
  # Limpa o restante do código que não faz parte do json
  resultado = resultado.split('},props:')[0].replace('};','}')
  resultado = json.loads(resultado)['initialState']['results']

  # Seleciona apenas os resultados que possuem as palavras chaves no título
  dados_analisados = []
  melhor_ratio = 0
  # Itera entre todos os resultados
  for r in resultado:
    ratio = []
    # Itera entre todas as palavras chaves
    for chave in palavras_chave:
      # Elimina os espaços e transfoma em minúsculo as duas frases a serem comparadas
      frase1 = r['title'].replace(' ','').lower()
      frase2 = chave.replace(' ','').lower()
      # Calcula a porcentagem de proximidade das frases
      ratio.append(fuzz.partial_ratio(frase1,frase2))
    ratio = round(sum(ratio)/len(ratio),0)
    dados_analisados.append((ratio,r))
    # Grava o recorde do ratio de proximidade
    if melhor_ratio < ratio: melhor_ratio = ratio

  # Monta uma lista com os resultados que obtiveram o maior ratio
  dados_selecionados = [x[1] for x in dados_analisados if x[0] == melhor_ratio]

  # Retorna a data da pesquisa e os resultados
  return (datetime.today().strftime('%d/%m/%y'),dados_selecionados)

Filtrando os dados necessários

In [15]:
def filtrar(items:tuple) -> dict:
  dados = {
      'Data':[],
      'Título':[],
      'Link':[],
      'id':[],
      'Preço':[],
  }
  for item in items[1]:
    dados['Data'].append(items[0])
    dados['Título'].append(item['title'])
    dados['Link'].append(item['permalink'])
    dados['id'].append(item['id'])
    dados['Preço'].append(item['price']['amount'])
  return dados

Testando

Para usar o script faça a uma pesquisa no mercado livre e copie o link da pesquisa na variável `url` e inclua palavras chaves dentro de uma lista na variável `palavras_chaves`.

As palavras chaves servem para selecionar apenas os resultados que se relacionam com essas palavras, isso é necessário, pois quando pesquisamos algum produto no Mercado Livre podemos ter como resultado variações do produto que são diferentes do que desejamos.

Ex.: Se eu pesquisar por iPhone 15 256GB o resultado pode incluir iPhones 13, iPhones 14, com capacidade de 128GB ou de 512GB e assim por diante. As palavras chaves definidas garantem que os resultados serão mais próximos ao desejado.

In [57]:
url = 'https://lista.mercadolivre.com.br/celulares-telefones/celulares-smartphones/novo/samsung-galaxy-a32-128gb_NoIndex_True#applied_filter_id%3DITEM_CONDITION%26applied_filter_name%3DCondi%C3%A7%C3%A3o%26applied_filter_order%3D6%26applied_value_id%3D2230284%26applied_value_name%3DNovo%26applied_value_order%3D1%26applied_value_results%3D29%26is_custom%3Dfalse'
palavras_chaves = ['Samsung Galaxy A32 128gb tela 6.5']
items = coletar_da_url(url,palavras_chaves)
items = filtrar(items)

Visualizando os dados

In [58]:
plot = px.box(items,y='Preço',title=' '.join(palavras_chaves))
plot.show()